Eres un analista de datos en una empresa de seguros médicos. Tu objetivo es desarrollar un modelo de **Regresión Lineal Múltiple** que permita predecir el **costo del seguro médico** (`charges`) en función de varias características de los clientes, como su edad, índice de masa corporal (IMC), número de hijos y si son fumadores o no.

El dataset `insurance.csv` (https://www.kaggle.com/datasets/mirichoi0218/insurance?resource=download) contiene la siguiente información:

- **`age`** → Edad del cliente.
- **`sex`** → Género (`male` o `female`).
- **`bmi`** → Índice de masa corporal (IMC).
- **`children`** → Número de hijos que tiene el cliente asegurado.
- **`smoker`** → Si el cliente es fumador (`yes/no`).
- **`region`** → Región geográfica donde reside el cliente (`northeast, northwest, southeast, southwest`).
- **`charges`** → Costo del seguro médico (variable objetivo).

Debes entrenar un modelo de **Regresión Lineal Múltiple** para predecir `charges` en función de las variables disponibles y evaluar su rendimiento con métricas como MAE, MSE y R².

### INSTRUCCIONES

1. **Cargar el dataset `insurance.csv`** desde la URL [pública o Kaggle**.**](https://www.kaggle.com/datasets/mirichoi0218/insurance?resource=download)
2. **Explorar los datos y convertir las variables categóricas (`sex`, `smoker`, `region`) a variables numéricas** mediante **One-Hot Encoding**.
3. **Seleccionar las variables predictoras (`X`) y la variable objetivo (`y`).**
4. **Escalar las variables numéricas** utilizando `StandardScaler()`.
5. **Dividir el dataset en conjuntos de entrenamiento (80%) y prueba (20%)** usando `train_test_split()`.
6. **Entrenar un modelo de Regresión Lineal** con `LinearRegression()` de `sklearn`.
7. **Obtener e interpretar los coeficientes y el intercepto del modelo.**
8. **Realizar predicciones con el conjunto de prueba (`X_test`).**
9. **Evaluar el modelo utilizando las métricas:**
    - **MAE (Error Absoluto Medio)**
    - **MSE (Error Cuadrático Medio)**
    - **RMSE (Raíz del Error Cuadrático Medio)**
    - **R² (Coeficiente de Determinación)**

### **PISTAS Y CONSEJOS**

- Para convertir las variables categóricas a numéricas, usa `pd.get_dummies()`.
- Al hacerlo, asegúrate de eliminar una categoría de cada variable categórica para evitar colinealidad (`drop_first=True`): `df = pd.get_dummies(df, drop_first=True)`
- Analiza el impacto de cada variable en la predicción revisando los coeficientes del modelo.
- Si el R² es bajo, ¿qué podrías hacer para mejorar el modelo?

In [1]:
import pandas as pd
import numpy as np 
from sklearn.discriminant_analysis import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
df = pd.read_csv("insurance.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [3]:
df

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [4]:
#Comprbacionde duplicados 
duplicados = df.duplicated()
print(f"Duplicados: {duplicados.sum()}")
df = df.drop_duplicates().reset_index()
df

Duplicados: 1


,index,age,sex,bmi,children,smoker,region,charges
0,0,19,female,27.900,0,yes,southwest,16884.92400
1,1,18,male,33.770,1,no,southeast,1725.55230
2,2,28,male,33.000,3,no,southeast,4449.46200
3,3,33,male,22.705,0,no,northwest,21984.47061
4,4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...,...
1332,1333,50,male,30.970,3,no,northwest,10600.54830
1333,1334,18,female,31.920,0,no,northeast,2205.98080
1334,1335,18,female,36.850,0,no,southeast,1629.83350
1335,1336,21,female,25.800,0,no,southwest,2007.94500


In [5]:
# Convertir una columna categórica en varias columnas binarias.
df["sex_hot"] = pd.get_dummies(df['sex'], drop_first=True)
df["smoker_hot"] = pd.get_dummies(df['smoker'], drop_first=True)
df_onehot_region = pd.get_dummies(df['region'], drop_first=True)
df

,index,age,sex,bmi,children,smoker,region,charges,sex_hot,smoker_hot
0,0,19,female,27.900,0,yes,southwest,16884.92400,False,True
1,1,18,male,33.770,1,no,southeast,1725.55230,True,False
2,2,28,male,33.000,3,no,southeast,4449.46200,True,False
3,3,33,male,22.705,0,no,northwest,21984.47061,True,False
4,4,32,male,28.880,0,no,northwest,3866.85520,True,False
...,...,...,...,...,...,...,...,...,...,...
1332,1333,50,male,30.970,3,no,northwest,10600.54830,True,False
1333,1334,18,female,31.920,0,no,northeast,2205.98080,False,False
1334,1335,18,female,36.850,0,no,southeast,1629.83350,False,False
1335,1336,21,female,25.800,0,no,southwest,2007.94500,False,False


In [6]:
X = df.drop(columns=["charges","region","sex","smoker","index", "sex_hot"])
y = df["charges"]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
print(f"Tamaño de X_scaled: {X_scaled.shape}, Tamaño de y: {y.shape}")

Tamaño de X_scaled: (1337, 4), Tamaño de y: (1337,)


In [7]:
df_scaled = pd.DataFrame(X_scaled, columns=X.columns)
df_scaled["charges"] = y

correlaciones = df_scaled.corr()["charges"].sort_values(ascending=False)

print("Correlaciones con la variable objetivo:")
print(correlaciones)

Correlaciones con la variable objetivo:
charges       1.000000
smoker_hot    0.787234
age           0.298308
bmi           0.198401
children      0.067389
Name: charges, dtype: float64


In [8]:
vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X_scaled, i) for i in range(X_scaled.shape[1])]

# Mostrar los resultados
print("VIF de cada variable:")
print(vif_data)


VIF de cada variable:
     Variable       VIF
0         age  1.014465
1         bmi  1.012213
2    children  1.001867
3  smoker_hot  1.000768


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)
modelo = LinearRegression()
modelo.fit(X_train, y_train)

# Obtener los coeficientes
coeficientes = pd.Series(modelo.coef_, index=X.columns)

# Mostrar el intercepto y los coeficientes
print("Intercepto:", modelo.intercept_)
print("Coeficientes:")
print(coeficientes)

Intercepto: 13189.343636937858
Coeficientes:
age           3498.416377
bmi           1861.577585
children       648.321319
smoker_hot    9301.226901
dtype: float64


In [10]:
y_pred = modelo.predict(X_test)

# Calcular métricas de evaluación
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Mostrar resultados
print(f"MAE (Error Absoluto Medio): {mae:.4f}")
print(f"MSE (Error Cuadrático Medio): {mse:.4f}")
print(f"RMSE (Raíz del Error Cuadrático Medio): {rmse:.4f}")
print(f"R² (Coeficiente de Determinación): {r2:.4f}")

MAE (Error Absoluto Medio): 4198.5926
MSE (Error Cuadrático Medio): 35914551.4804
RMSE (Raíz del Error Cuadrático Medio): 5992.8751
R² (Coeficiente de Determinación): 0.8046
